# Lesson 3 - Configure and Commit

## Live Methods

In the last lesson you learned to add objects to a configuration tree and "create" them on the firewall using the `create()` method. In this lesson we'll dig into more complex configurations like security policy and interfaces.

First, lets introduce the Live Methods.  Like `create()` the Live Methods are any method that touches the live device.  Device Framework allows you to create a complete configuration in memory before ever touching a live firewall or Panorama. When you want to make the changes on the live device, you use the Live Methods:

| Live Method      | Description                                                                        |
|------------------|------------------------------------------------------------------------------------|
| apply            | Change the object on the live device (or create if it doesn't exist)               |
| create           | Create an object on the live device (or merge if it already exists)                |
| delete           | Delete the object on the live device                                               |
| update           | Update a single parameter of an object on the live device                          |
| refresh          | Pull the current state of an object from the live device                           |
| refreshall       | Create objects in python to represent all the objects of a type on the live device |
| refresh_variable | Pull the current state of a single parameter of an object from the live device     |

The "refresh" methods above pull configuration from the live device into the python configuration tree.  The other methods push configuration from the python configuration tree to the live device. The live methods we we'll focus on for this lab are:

- apply
- create
- delete
- refresh
- refreshall

## Create objects

Let's give some of these Live Methods a try by creating 2 ethernet interfaces:

In [ ]:
# First, import the needed modules and create a Firewall object to work with
from pandevice import firewall, objects, network
fw = firewall.Firewall('10.30.11.101', 'admin', 'Ignite18')

In [ ]:
# Then, create 2 ethernet interfaces and add them to the config tree
eth3 = network.EthernetInterface('ethernet1/3', ip='10.0.0.101', comment='Connects to Switch1')
eth4 = network.EthernetInterface('ethernet1/4', ip='10.0.0.102')
fw.add(eth3)
fw.add(eth4)

In [ ]:
# Lets display the tree to see what we have so far
fw.tree()

In [ ]:
# Last, push the ethernet interfaces to the live firewall one at a time
# (you can push them all in bulk, but we'll cover that in another lesson)
eth3.create()
eth4.apply()

Check the firewall GUI to verify the ethernet interfaces were created (**Network tab -> Interfaces**). Notice that we used `create()` and `apply()` to create the ethernet interfaces. This was to demonstrate that when an object doesn't exist on the live device yet, these methods have the same behavior. They both create the object on the firewall.

## Modify objects

When an object already exists on the firewall, `create()` and `apply()` will behave differently.  The `create()` method will merge the object in memory with the object on the live device, non-destructively. It only "creates" configuration, never removes it. The `apply()` method will make the object on the live device match the object in memory exactly.

Objects have parameters, like `ip`, `description`, or `comment`. The difference between `apply()` and `create()` is clear when you remove a parameter from an object. To remove a parameter, set the parameters value to `None`. Then an `apply()` on the object would remove the parameter from the live device, while a `create()` would not.

It is generally safest to use `apply()` when you don't know if an object exists or not on the live device because `create()` could leave leftover parameters in the object on the live device.

## Exercise 1: Modify an ethernet interface

We created `eth3` in the last section. For this exercise, you'll make a couple changes to `eth3`.

Change the **IP** of `eth3` to 10.0.0.103 and remove its **comment** parameter. Use `eth3.about()` in the yellow cell to validate the parameters of `eth3`. When you're satisfied with your changes in the python object, push the changes to the live device using the appropriate cell below (the one after the yellow `eth3.about()` cell).

**Remember, `eth3` already exists in python and the live firewall. Use the `eth3` variable created above in the examples. You don't need to create a new Firewall object or import pandevice in this exercise since that was already done earlier.**

In [ ]:
# make changes in python memory
eth3.ip = '10.0.0.103'
eth3.comment = None

# use about to check your work
eth3.about

# then apply the changes to the live firewall
eth3.apply()

In [ ]:
# Exercise 1: modify eth3 here...




In [ ]:
# Use this to validate the parameters of eth3
eth3.about()

Use `eth3.about()` in the previous cell to validate your EthernetInterface object before sending it to the firewall. In the next cell, push the changes to the live firewall.

In [ ]:
# Exercise 1: push changes to the live firewall here...


**Validate** your answer using the firewall GUI to see that your changes took effect on the live firewall.

## Exercise 2: Delete an ethernet interface

Delete `eth4` from the firewall. Use the firewall GUI to verify the delete worked.


**Once again, use the existing `eth4` variable created in the examples above. You don't need to create a new Firewall object or import pandevice in this exercise since that was already done earlier.**

Hint: The solution to this exercise fits in one line. You won't need `apply()` for this, you'll need a different live method.

In [ ]:
eth4.delete()

In [ ]:
# Exercise 2 answer here...


Validate your answer by checking the configuration tree. The ethernet interface called `ethernet1/4` should be gone.

In [ ]:
fw.tree()

Notice from the config tree that `eth4` wasn't just removed from the firewall, it was removed from the configuration tree.  It still exists in memory (you can see it with `eth4.about()`, but it won't be part of the config tree unless you `add()` it back in.

## References

Objects can reference other objects, such as an address group that groups address objects together. The address group references the address objects. To create a reference in Device Framework, pass in the object being referenced or the name of the object being reference, either works.

This example creates an AddressGroup that groups (references) two AddressObjects:

In [ ]:
# Create two address objects and add them to the tree
obj1 = objects.AddressObject('MyServer1', '10.1.1.1')
fw.add(obj1)

# We'll use a shorthand for this one to create the
# AddressObject and add it to the tree in one step.
obj2 = fw.add(objects.AddressObject('MyServer2', '10.0.0.2'))

# And check the tree
fw.tree()

In [ ]:
# Apply the AddressObjects to the live firewall
obj1.apply()
obj2.apply()

In [ ]:
# Now create an AddressGroup to group the AddressObjects.
group1 = objects.AddressGroup('MyServers', [obj1, obj2])
fw.add(group1)
group1.apply()

In this example, we passed a list containing the two address objects to the AddressGroup. We could also have passed a list of names of the objects: `['MyServer1', 'MyServer2']`

Verify the address group was created in the firewall GUI under **Objects -> Address Groups**.

## Refresh

At the beginning of this lesson there was a table of Live Methods which included several "refresh" methods. The methods we've used so far have all pushed configuration to the live firewall, but what if we want to pull configuration from the live firewall into the python configuration tree? That's what the "refresh" methods are for.

The `refreshall()` method is very useful because it pulls all the object of a type from the live firewall. You may have noticed in the firewall GUI there are more ethernet interfaces on the firewall that the ones we created. So why don't those interfaces show up in our configuration tree? The tree was built by you using `add()` so it isn't aware of the interfaces previously on the firewall. This separation allows you limit the scope of your changes. But many times we want all the objects from a firewall so we know the full context of our changes, so we'll refresh our configuration tree with all the ethernet interfaces from the firewall.

First, we'll remove `eth3` from the Firewall object. This doesn't delete it from the live firewall (you would call `delete()` to do that). It just removes it from the configuration tree so we don't have any EthernetInterface objects in the tree.

In [ ]:
fw.remove(eth3)
fw.tree()

You can see from the `tree()` method that our Firewall has no EthernetInterface child objects. This time, instead of creating the child objects yourself, you'll create the child objects from the live device. Because the `refreshall()` method creates new objects, you can't call it on any specific object. Call the `refreshall()` method on the **type** of object you want to refresh, and pass in the **parent** object as an argument (in this case, the Firewall object).

In [ ]:
network.EthernetInterface.refreshall(fw)

The method returns a list of the objects pulled from the firewall, and also adds the objects as children of the parent object you passed in (the Firewall).

Verify this with the `tree()` method.

In [ ]:
fw.tree()

You can also access the children of any node.

In [ ]:
fw.children

In this case the children are the same list of EthernetInterfaces that was returned by the `refreshall()` method plus any children that were already there.

You can find a specific child by name with the `find()` method:

In [ ]:
eth3 = fw.find('ethernet1/3')
eth3.ip

## Exercise 3: Refresh All Security Policies

Refresh all the current security policies from the live firewall. This isn't quite like refreshing all EthernetInterfaces because there is an intermediate object call `Rulebase`. Check out the [Firewall Config Tree Diagram](http://pandevice.readthedocs.io/en/latest/configtree.html#firewall) and you'll see that a Rulebase can be added to a Firewall object, but a SecurityRule cannot. In the [Policy Config Tree](http://pandevice.readthedocs.io/en/latest/configtree.html#firewall) it shows that a NatRule and SecurityRule can be a child of a Rulebase.

So here's the steps you'll need to take:

1. Import the 'policies' module from pandevice
2. Create a `policies.Rulebase` object and add it as a child of `fw`
3. Call `refreshall()` on `policies.SecurityRule`, with the rulebase you created as an argument

In [ ]:
from pandevice import policies
rulebase = fw.add(policies.Rulebase())
policies.SecurityRule.refreshall(rulebase)

In [ ]:
# Exercise 3 answer here...


In [ ]:
fw.tree()

**Verify** your answer using `fw.tree`.  You should see a Rulebase object with two SecurityRule children objects.

## Commit

We've made a lot of changes, and none of them will take effect until they're committed. So lets run a commit.

Commit is an asynchronous operation, meaning when you request a commit the firewall starts a job that could take some time and shouldn't block other simultaneous operations. So the main thing to consider when committing from Device Framework is if your python program should wait for the commit to finish before continuing. This is called a synchronous commit because it stops the python program until the commit finishes.

- **Asynchronous commit:** Continue to the next python command without waiting (default)
- **Synchronous commit:** Stop and wait for the commit to finish, then get the result of the commit and continue

Commits are executed by calling the `commit()` method on a Firewall or Panorama object. To wait for the commit to finish and return the result (synchronous commit) pass in the `sync` argument: `commit(sync=True)`.

Perform a synchronous commit of our changes from this lesson. Because we are waiting for the commit to finish, you will see a `[*]` to the left of the command while the commit is running. You can monitor the progress of the commit in the firewall GUI in the **Tasks** menu (bottom right).

In [ ]:
fw.commit(sync=True)

When the commit finishes you'll see a 'success' boolean and any errors or warnings from the commit. You'll likely have a couple warnings about 'ethernet1/3' because we never set the zone or virtual-router for that interface after we created it.

For extra-credit, configure a Zone object and VirtualRouter object and reference the 'ethernet1/3' EthernetInterface object to resolve these warnings.

Or continue on to [Lesson 4 - User-ID](Lesson 4 - User-ID.ipynb)